<a href="https://colab.research.google.com/github/hits-sdo/hits-sdo-similaritysearch/blob/byol-training_pipeline/search_byol/byol_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook to Intialize HITS-SDO self-similarity search environment
- Run all cells to initalize environment, and restart runtime if prompted to use updated versions. You will need to rerun the cells again to ensure that all dependencies have been installed.

# Download and Unzip Data

In [1]:
# Download Data
!gdown 15C5spf1la7L09kvWXll2qt67Ec0rwLsY

Downloading...
From: https://drive.google.com/uc?id=15C5spf1la7L09kvWXll2qt67Ec0rwLsY
To: /content/aia_171_color_1perMonth.tar.gz
100% 146M/146M [00:02<00:00, 72.5MB/s]


In [2]:
# Unzip file
!tar -zxf aia_171_color_1perMonth.tar.gz

In [3]:
# Print some files to see that they exist
!du aia_171_color_1perMonth/. -l -h

3.2M	aia_171_color_1perMonth/./20220704_000036_aia.lev1_euv_12s_4k/tiles
40K	aia_171_color_1perMonth/./20220704_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./20220704_000036_aia.lev1_euv_12s_4k
3.2M	aia_171_color_1perMonth/./20190712_000036_aia.lev1_euv_12s_4k/tiles
40K	aia_171_color_1perMonth/./20190712_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./20190712_000036_aia.lev1_euv_12s_4k
3.2M	aia_171_color_1perMonth/./20210412_000036_aia.lev1_euv_12s_4k/tiles
40K	aia_171_color_1perMonth/./20210412_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./20210412_000036_aia.lev1_euv_12s_4k
3.2M	aia_171_color_1perMonth/./20101210_000036_aia.lev1_euv_12s_4k/tiles
40K	aia_171_color_1perMonth/./20101210_000036_aia.lev1_euv_12s_4k/tile_meta_data
3.2M	aia_171_color_1perMonth/./20101210_000036_aia.lev1_euv_12s_4k
3.2M	aia_171_color_1perMonth/./20200425_000036_aia.lev1_euv_12s_4k/tiles
40K	aia_171_color_1perMonth/./20200425_0000

# Clone repository

In [20]:
# Clone the repository from GitHub
!git clone https://github.com/hits-sdo/hits-sdo-similaritysearch

Cloning into 'hits-sdo-similaritysearch'...
remote: Enumerating objects: 426, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 426 (delta 15), reused 16 (delta 12), pack-reused 384
Receiving objects: 100% (426/426), 6.89 MiB | 15.62 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [21]:
%cd hits-sdo-similaritysearch/

/content/hits-sdo-similaritysearch/hits-sdo-similaritysearch/hits-sdo-similaritysearch/hits-sdo-similaritysearch


# Switch to Desired Branch

In [22]:
# Switch to the desired branch with requirements.txt
!git checkout byol-training_pipeline

Branch 'byol-training_pipeline' set up to track remote branch 'byol-training_pipeline' from 'origin'.
Switched to a new branch 'byol-training_pipeline'


In [23]:
# Confirm that branch is up to date
!git log --oneline

cb8037b (HEAD -> byol-training_pipeline, origin/byol-training_pipeline) fix opencv pip repository in requirements
0db1aa6 add a version requirement to opencv
0dd5485 Merge branch 'byol-training_pipeline' of https://github.com/hits-sdo/hits-sdo-similaritysearch into byol-training_pipeline
4183c2c Add opencv to requirements
7b7e31d Add link to colab
e99626e Re-commit training notebook
464356e Initalize BYOL training notebook
f6a7bcf (origin/byol-augmentation) Rename database class and add augmentations
6b23428 (origin/byol-loader) Initialize database and its test suite
d83a4b3 Add npy files to gitignore
acdcd3c Merge pull request #1 from hits-sdo/initialization
b6b76a7 (origin/initialization) Make main the default branch in the notebook setup
3e41ea2 Added download to setup notebook
7135c8a replaced 'search-utils' with 'search_utils'
8c2014c replaced '-' with '_' in module names
97a1002 added search_utils to sys.path
dd9c215 added search-utils to sys.path
0702056 renamed folder module ut

# Install all necesary packages into environment

In [24]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hits-sdo/hits-sdo-packager.git (to revision pip_nodata) to /tmp/pip-req-build-mphqn98v
  Running command git clone --filter=blob:none --quiet https://github.com/hits-sdo/hits-sdo-packager.git /tmp/pip-req-build-mphqn98v
  Running command git checkout -b pip_nodata --track origin/pip_nodata
  Switched to a new branch 'pip_nodata'
  Branch 'pip_nodata' set up to track remote branch 'pip_nodata' from 'origin'.
  Resolved https://github.com/hits-sdo/hits-sdo-packager.git to commit 3a54caba2ae6bf7caf4eabaf580a6ffda1b3bbda
  Preparing metadata (setup.py) ... done


##  Load Modules

In [29]:
import copy

import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset
from lightly.data.multi_view_collate import MultiViewCollate
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import BYOLPredictionHead, BYOLProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.simclr_transform import SimCLRTransform
from lightly.utils.scheduler import cosine_schedule

from search_byol.database import SDOTilesDataset

## Define BYOL Model

In [26]:
class BYOL(nn.Module):
    def __init__(self, backbone):
        super().__init__()

        self.backbone = backbone
        self.projection_head = BYOLProjectionHead(512, 1024, 256)
        self.prediction_head = BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        z = z.detach()
        return z

## Initialize Module

In [27]:
resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = BYOL(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BYOL(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

## Initialize Dataloader

In [31]:
data_path = '/content/aia_171_color_1perMonth'
dataset = SDOTilesDataset(data_path=data_path, double_augmentation=False)
# or create a dataset from a folder containing images or videos:
# dataset = LightlyDataset("path/to/folder")


dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    drop_last=True,
    num_workers=2,
)

## Run Training Loop 

In [34]:
criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

epochs = 10

print("Starting Training")
for epoch in range(epochs):
    total_loss = 0
    momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
    for (x0, x1) in dataloader:
        update_momentum(model.backbone, model.backbone_momentum, m=momentum_val)
        update_momentum(
            model.projection_head, model.projection_head_momentum, m=momentum_val
        )
        print(x0.shape)
        x0 = x0.to(device)
        x1 = x1.to(device)
        p0 = model(x0)
        z0 = model.forward_momentum(x0)
        p1 = model(x1)
        z1 = model.forward_momentum(x1)
        loss = 0.5 * (criterion(p0, z1) + criterion(p1, z0))
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(dataloader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

Starting Training
torch.Size([256, 128, 128, 3])


RuntimeError: ignored